In [7]:
import pandas as pd 
import numpy as np 

In [8]:
df = pd.read_csv('../Datasets/train.csv')
df.head()

,period,gasoline_imports,gasoline_exports,crude_imports,crude_exports,gasoline_stocks,crude_stocks,distillate_fuel_stocks,jet_stocks,propane_stocks,...,propane_output,residual_fuel_output,conventional_gasoline_spot_price,crude_brent_spot_price,gasoline_future_price1,gasoline_future_price2,gasoline_future_price3,gasoline_future_price4,gasoline_demand,target
0,2010-10-15,779,200,8600,33,219329,1066948,170055,47088,63549,...,1044,414,2.202,83.11,2.139,2.119,2.128,2.146,9460,10639
1,2010-10-22,999,200,9463,33,214942,1071996,168442,46829,64284,...,1021,473,2.146,81.43,2.077,2.063,2.076,2.097,10639,10036
2,2010-10-29,871,200,8578,33,212253,1073946,164874,45830,64469,...,1032,401,2.122,82.25,2.098,2.072,2.085,2.106,10036,9886
3,2010-11-05,802,200,8089,33,210336,1070399,159902,45940,63833,...,1004,464,2.174,85.60,2.140,2.149,2.170,2.193,9886,9706
4,2010-11-12,562,167,7864,34,207679,1063113,158792,44817,64776,...,1006,417,2.242,87.43,2.209,2.194,2.207,2.226,9706,9286


In [9]:
highly_correlated_cols = ['conventional_gasoline_spot_price',
       'crude_brent_spot_price', 'gasoline_future_price1',
       'gasoline_future_price2', 'gasoline_future_price3',
       'gasoline_future_price4']

In [10]:
highly_df = df[highly_correlated_cols].copy()

for col in highly_correlated_cols:
    highly_df[col] = (highly_df[col] - highly_df[col].mean())/highly_df[col].std()

In [11]:
import plotly.express as px 

fig = px.imshow(np.round(highly_df.corr(),2), text_auto=True, aspect='auto')
fig.update_layout(title='Correlation Matrix of Highly Correlated Columns', height=800, title_font_size=30, title_x=.5)
fig.show()

<h3>PCA uygulanabilir mi?</h3>  

PCA Yüksek korelasyonlu değişkenleri daha az değişkenlere indirgemek için kullanılan bir tekniktir.  

$$ X^{2}_{Hesap} = -[(n-1)-\frac{1}{6}(2p+5)]ln(\mid R \mid)$$  

$$ X^{2}_{Tablo} = X^{2}_{\alpha;\frac{p(p-1)}{2}} $$  

<p align="center">
(p: Değişken Sayısı)  

$H_0:$ Değişkenler arası ilişki istatistiksel olarak anlamsızdır.  
$H_1:$ Değişkenler arası ilişki istatistiksel olarak anlamlıdır.  
</p>

Eğer $X^2_{Hesap}$ değeri $X^2_{Tablo}$ değerinden büyük çıkarsa yokluk hipotezi reddedilir ve PCA yapılabilir şeklinde yorumlanabilir.

In [12]:
import math

R = highly_df.corr().values
R_det = np.linalg.det(np.array(R))

chi_H = -(((len(df)-1) - 1/6*(2*(len(df.columns)+5)))*math.log(R_det))
chi_H

10237.206399825174

$\alpha=0.05$  
Serbestlik derecesi: Değişken sayısı - 1 = 14(14-1)/2 = 91  
$X^2_{Tablo}=X^2_{0.05;91} = 122.43$  

$X^2_{Hesap} > X^2_{Tablo}$ için $H_0$ reddedilir. %95 Güvenle söylenebilir ki PCA yapılabilir.   

### Principal Component Analysis (PCA)  
1. Component Score Matrix ($\lambda$)

In [13]:
from numpy import linalg as LA

lambdas, T = LA.eig(np.array(R))
pd.DataFrame(lambdas).T

,0,1,2,3,4,5
0,5.906544,0.045868,0.033359,0.008437,0.00356,0.002232


$\lambda_i > 1.0$ için önemli boyut sayısı 1'dir.  

2. Eigen Vectors (T)

In [14]:
T = pd.DataFrame(T)
T

,0,1,2,3,4,5
0,0.407667,0.559492,0.058851,0.651757,-0.292396,0.083847
1,0.406718,0.156642,0.775369,-0.456811,0.000873,-0.013019
2,0.408663,0.356305,-0.417937,-0.211556,0.588435,-0.374643
3,0.409805,-0.055107,-0.407303,-0.365460,-0.232987,0.689408
4,0.409242,-0.418824,-0.167109,-0.047332,-0.536862,-0.581997
5,0.407386,-0.597518,0.163850,0.431253,0.475125,0.196058


3. Component Matrix (V):  

$V_i = \sqrt{\lambda_i}\cdot T_i$

In [15]:
V = pd.DataFrame()
for i in T:
    V[f"V{i+1}"] = np.sqrt(lambdas[i])*T[i]
V

,V1,V2,V3,V4,V5,V6
0,0.990769,0.119825,0.010749,0.059867,-0.017447,0.003961
1,0.988463,0.033548,0.141617,-0.041960,0.000052,-0.000615
2,0.993190,0.076309,-0.076334,-0.019432,0.035111,-0.017699
3,0.995965,-0.011802,-0.074392,-0.033569,-0.013902,0.032568
4,0.994596,-0.089699,-0.030522,-0.004348,-0.032034,-0.027494
5,0.990086,-0.127969,0.029926,0.039612,0.028350,0.009262


* Temel bileşen matrisi (component matrix) mutlak değerce yorumlanmalıdır.  
* Önemli bileşen sayısı kadar en iyi vektör seçilir. En iyi vektörden kasıt değişkenlerimizi en çok açıklayabilen vektördür.

In [16]:
V = V.abs()
pd.DataFrame(np.round(V.mean(),2)).T

,V1,V2,V3,V4,V5,V6
0,0.99,0.08,0.06,0.03,0.02,0.02


V1 vektörü veri setimizi %99 oranında temsil edebiliyormuş. Bu çok iyi bir yüzde. Bu demektir ki birbiri ile yüksek koreleye sahip bu veri setini tek bir değişkene indirgeyebiliriz ve bu tek değişken orjinal veri setindeki değişimin %99'unu açıklayabiliyor.

In [17]:
v1 = V.iloc[:,0]
pd.DataFrame(v1).T

,0,1,2,3,4,5
V1,0.990769,0.988463,0.99319,0.995965,0.994596,0.990086


4. Cumulanities (W)  
$W = V_{ij}^2$

In [18]:
W = V**2
W

,V1,V2,V3,V4,V5,V6
0,0.981622,0.014358,0.000116,0.003584,3.043916e-04,1.568987e-05
1,0.977058,0.001125,0.020055,0.001761,2.711281e-09,3.782879e-07
2,0.986426,0.005823,0.005827,0.000378,1.232780e-03,3.132406e-04
3,0.991946,0.000139,0.005534,0.001127,1.932649e-04,1.060707e-03
4,0.989222,0.008046,0.000932,0.000019,1.026156e-03,7.559361e-04
5,0.980270,0.016376,0.000896,0.001569,8.037191e-04,8.578524e-05


Yukarıdaki formülden Component Matrix ‘in (V)her bir değişkeninin karesi alınarak bulunur.

* Sütun toplamının değişken sayısına bölümü temel bileşenlerin değişkenlerdeki toplam değişimin açıklanma oranını söyler.
* Satır toplamının değişken sayısına bölümü ise, temel bileşenlerin değişkenleri açıklama oranını verir.  

Örneğin W matrisinin 1. satır toplamı, temel bileşenlerin tüm veri setinin % kaçını açıklıyor bunu gösterirken sütun toplamı ise temel bileşenlerin, değişkenlerin toplam değişiminin % kaçını açıklıyor bunu gösterir.

Biz önceki kısımda V1 i veri setimizi en iyi temsil edebilecek temel bileşen vektörü olarak seçmiştik, o zaman V1 sütunun toplamını p ‘ye (değişken sayısı) bölümü bize temel bileşenlerimizin toplam değişiminin % kaçını açıkladığını gösterir:  

In [19]:
W['V1'].sum()/len(W.columns)

0.9844239487293677

Bu kısımda söyleyebiliriz ki V1 vektörünün temel bileşenleri bizim veri setimizdeki değişkenlerin toplam değişiminin %98.44 ‘sini açıklayabiliyor.

5. Temel Bileşenlerin Veri Setine Uyarlanması (Y):  

$T_1:$ Önemli temel bileşen vektörünün veri setindeki 1. değişkeni temsil eden 1. değer.  
$Z_1:$ 1. Sütundaki verilerin standartlaştırılmış hali.  

$Y = \sum{T_i Z_i}$

Yukarıdaki formülde verildiği gibi her bir sütundaki standartlaştırılmış veriyi seçtiğimiz öz vektörün o sütuna denk gelen indexi ile çarpalım.  
Yani eğer önemli temel bileşen sayımızı 1 adet seçtiysek, öz vektörümüzün 1. vektörünün 1. değerini `conventional_gasoline_spot_price` değişkeni ile çarpalım.  


In [20]:
T

,0,1,2,3,4,5
0,0.407667,0.559492,0.058851,0.651757,-0.292396,0.083847
1,0.406718,0.156642,0.775369,-0.456811,0.000873,-0.013019
2,0.408663,0.356305,-0.417937,-0.211556,0.588435,-0.374643
3,0.409805,-0.055107,-0.407303,-0.365460,-0.232987,0.689408
4,0.409242,-0.418824,-0.167109,-0.047332,-0.536862,-0.581997
5,0.407386,-0.597518,0.163850,0.431253,0.475125,0.196058


In [21]:
T[0]

0    0.407667
1    0.406718
2    0.408663
3    0.409805
4    0.409242
5    0.407386
Name: 0, dtype: float64

In [22]:
highly_df.columns.get_loc('conventional_gasoline_spot_price')

0

In [23]:
T[0][0]

0.40766694089861916

In [24]:
highly_df[['conventional_gasoline_spot_price']].head()

,conventional_gasoline_spot_price
0,0.084170
1,-0.006790
2,-0.045773
3,0.038690
4,0.149142


In [25]:
for col in highly_df.columns:
    highly_df[col] = T[0][highly_df.columns.get_loc(col)]*highly_df[col]
highly_df.head()

,conventional_gasoline_spot_price,crude_brent_spot_price,gasoline_future_price1,gasoline_future_price2,gasoline_future_price3,gasoline_future_price4
0,0.034313,0.059317,-0.009028,-0.015828,-0.005728,0.009942
1,-0.002768,0.034219,-0.049114,-0.052937,-0.040985,-0.023860
2,-0.018660,0.046469,-0.035537,-0.046973,-0.034883,-0.017651
3,0.015773,0.096515,-0.008382,0.004053,0.022749,0.042363
4,0.060800,0.123854,0.036230,0.033873,0.047836,0.065128


Bu çarpımlarımızın sonucunda her satırın sonucunu sütun bazlı toplayıp Y (PCA1) sütununa yazmalıyız.

In [26]:
highly_df['PCA1'] = highly_df.sum(axis=1)
highly_df.head()

,conventional_gasoline_spot_price,crude_brent_spot_price,gasoline_future_price1,gasoline_future_price2,gasoline_future_price3,gasoline_future_price4,PCA1
0,0.034313,0.059317,-0.009028,-0.015828,-0.005728,0.009942,0.072988
1,-0.002768,0.034219,-0.049114,-0.052937,-0.040985,-0.023860,-0.135445
2,-0.018660,0.046469,-0.035537,-0.046973,-0.034883,-0.017651,-0.107235
3,0.015773,0.096515,-0.008382,0.004053,0.022749,0.042363,0.173071
4,0.060800,0.123854,0.036230,0.033873,0.047836,0.065128,0.367720


In [27]:
highly_df[['PCA1']].head()

,PCA1
0,0.072988
1,-0.135445
2,-0.107235
3,0.173071
4,0.367720


### PCA Function

In [28]:
highly_corr_cols = ['conventional_gasoline_spot_price',
       'crude_brent_spot_price', 'gasoline_future_price1',
       'gasoline_future_price2', 'gasoline_future_price3',
       'gasoline_future_price4']

In [44]:
def get_pca_n_components(df,highly_corr_cols, threshold):
    # highly correlated columns
    highly_df = df[highly_corr_cols].copy()
    
    for col in highly_df.columns:
        highly_df[col] = (highly_df[col] - highly_df[col].mean())/highly_df[col].std()
    
    # correlation matrix
    R = highly_df.corr().values
    
    # eigen vectors 
    lambdas, T = LA.eig(np.array(R))
    T = pd.DataFrame(T)
    
    # component matrix
    V = pd.DataFrame()
    for i in T:
        V[f"V{i+1}"] = np.sqrt(lambdas[i])*T[i]
    V = V.abs()
    
    Thresholder = pd.DataFrame(np.round(V.mean(),2)).T
    
    threshold_value = 0
    threshold_counter = 0
    
    for col in Thresholder:
        if threshold_value > threshold:
            break
        else:
            threshold_value += Thresholder[col].values[0]
            threshold_counter += 1
    
    n_component = threshold_counter
    
    return V, T, threshold_counter, threshold_value